# The Embedded Pseudo-Fragment Approach: 1D Carbon Chain

In this tutorial you will learn how to use the embedded pseudo-fragment approach of BigDFT, which generates optimised support functions for a template which is _embedded_ in a representative system, and then uses them as a fixed basis for a larger system. In this tutorial, we consider both a periodic and finite chain of carbon atoms.

This tutorial follows the methodology described in L. E. Ratcliff and L. Genovese, J. Phys.: Condens. Matter 31, 285901 (2019).

This tutorial follows on from the tutorial on molecular fragments, and thus also has the same prerequisites.


Notebook by Dr. Laura Ratcliff. 

In [1]:
install = "client (Google drive)" #@param ["full_suite", "client (Google drive)", "client"]
install_var=install
!wget https://gitlab.com/luigigenovese/bigdft-school/-/raw/main/packaging/install.py &> /dev/null
args={'locally': True} if install == 'client' else {}
import install
getattr(install,install_var.split()[0])(**args)

Mounted at /content/drive
Executing: mkdir -p /content/drive/MyDrive

Executing: git clone --depth 1 https://github.com/BigDFT-group/bigdft-school
Error Occurred:  
 fatal: destination path 'bigdft-school' already exists and is not an empty directory.

Executing: mkdir -p /content/drive/MyDrive/bigdft-school



In [3]:
install.data('data/carbon_chain.tar.gz')

## Periodic Carbon Chain

### System Construction

In order to demonstrate the differences compared to the molecular fragment approach, first we will consider a simple model system, namely a periodic chain of C atoms. Using the tools introduced in the System Generation tutorial, we'll first construct the system.

In [4]:
from BigDFT.Systems import System
from BigDFT.UnitCells import UnitCell
from BigDFT.Fragments import Fragment
from BigDFT.Atoms import Atom

# let's define a function so we can easily reuse it
def create_periodic_chain(bond_length, nat):
    
    # first we create a system
    periodic_chain = System()

    # and calculate the cell length
    cell_length = nat * bond_length

    # using this, we can construct a system, where each atom will be its own fragment
    for iat in range(nat):
        atom = Atom({"sym": "C", "r": [0.0, 0.0, iat * bond_length], "units": "angstroem"})
        periodic_chain["FRA:"+str(iat)] = Fragment([atom])

    # the chain is aligned along the z-axis, so we can use 1D boundary conditions
    periodic_chain.cell = UnitCell([float("inf"), float("inf"), cell_length], units="angstroem")
    
    return periodic_chain


# we'll now define some parameters for our chain
bond_length = 1.5
nat = 20
periodic_chain = create_periodic_chain(bond_length, nat)

Let's take a look at the system.

In [5]:
from BigDFT.Visualization import InlineVisualizer, get_colordict

viz = InlineVisualizer(400, 100)
cdict = get_colordict(periodic_chain, field_vals=[float(x) for x in range(len(list(periodic_chain)))], 
                      colorcode="rainbow")
viz.display_system(periodic_chain, zoom=3.0, colordict=cdict)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Isolated C Atom Template

Since each atom in the chain is identical (to within the error induced by the egg-box effect), we can define a single fragment type, consisting of a single carbon atom, so let's also construct a single C atom system.

Once we've created the system, we'll also write it to an xyz file, since we will need this for the fragment calculation.

In [6]:
from BigDFT.IO import write_xyz

template = System()
atom = Atom({"sym": "C", "r": [0.0, 0.0, 0.0], "units": "angstroem"})
template["FRA:0"] = Fragment([atom])

template_name = 'isolated_C'
ofile = open(template_name+'.xyz', 'w')
write_xyz(template, ofile)
ofile.close()

So far, we only know how to generate isolated templates. We expect this will be a poor approximation, since the C atoms are covalently bonded, but let's try anyway to see what happens. We can use the same input file parameters as for molecular fragments.

In [7]:
from BigDFT import Inputfiles as I, InputActions as A

# template input file
inpt = I.Inputfile()

# fragment input file
inpf = I.Inputfile()

# common parameters
for inp in [inpt, inpf]:
    # set the grid spacing, XC functional and specify pseudopotential type
    inp.set_hgrid(0.5)
    inp.set_xc('PBE')
    inp.set_psp_nlcc()
    
    # import the linear scaling profile
    inp['import'] = 'linear'
    
    # turn off the communications checks, which can be costly for larger systems
    inp.update({'perf': {'check_sumrho': 0, 'check_overlap': 0}})
    
    # output the SFs in binary format, and skip the multipole calculation
    inp.update({'lin_general': {'output_wf': 2, 'charge_multipoles': 0}})

We'll set up the calculator using only a few MPI tasks, as it doesn't make sense to have lots of MPI tasks for a single atom.

In [8]:
from BigDFT import Calculators as C
code = C.SystemCalculator(omp=2, mpi_run='mpirun -n 2', verbose=False, skip=True)

And we'll again define our functions to check convergence.

In [9]:
def check_convergence_optimised_sfs(run):
    
    # in this case we check the convergence of the outer loop
    dout = run.log['Ground State Optimization'][-1]['self consistency summary'][-1]['delta out']

    # check what threshold we were trying to meet
    dthresh = float(run.log['lin_general']['rpnrm_cv'])
    
    if dout > dthresh:
        print('WARNING, calculation did not converge')
        return False
    else:
        return True
    
def check_convergence_fixed_sfs(run):
    
    # in this case we check the convergence of the kernel loop
    dout = run.log['Ground State Optimization'][-1]['kernel optimization'][-1]['summary']['delta']

    # check what threshold we were trying to meet
    dthresh = float(run.log['lin_kernel']['rpnrm_cv'])
    
    if dout > dthresh:
        print('WARNING, calculation did not converge')
        return False
    else:
        return True

Now let's run the isolated C atom template.

In [10]:
run_template = code.run(input=inpt, sys=template, name=template_name)                

converged = check_convergence_optimised_sfs(run_template)

Ha2eV = 27.211396132
template_energy = Ha2eV * run_template.energy / run_template.nat
template_time = run_template.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Isolated template took '+'{0:.1f}'.format(template_time)+' minutes, E = '+\
      '{0:.3f}'.format(template_energy)+' eV/atom')

Isolated template took 0.4 minutes, E = -169.630 eV/atom


As in the molecular fragments tutorial, let's also run the chain using full linear scaling mode, to give us a reference. We will make the same modifications to the input file as before.

Before we run, we'll also change the calculator - now that we have more atoms, we can also use more MPI tasks.

In [11]:
code = C.SystemCalculator(omp=2, mpi_run='mpirun -n 4', verbose=False, skip=True)

# we'll name the calculation after the number of atoms in the chain
name = 'periodic_chain'+str(nat)+'_linear'

inpt['lin_general'].update({'output_wf': 0, 'output_mat': 0})
inpt['lin_general'].update({'subspace_diag': True})

run_full = code.run(input=inpt, sys=periodic_chain, name=name)    

converged = check_convergence_optimised_sfs(run_full)

full_energy = Ha2eV * run_full.energy / run_full.nat
full_time = run_full.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Fully optimised chain took '+'{0:.1f}'.format(full_time)+' minutes, E = '+\
      '{0:.3f}'.format(full_energy)+' eV/atom')

Fully optimised chain took 1.7 minutes, E = -176.777 eV/atom


In order to run the fragment calculation, as before, we also then need to setup the data directories. Since we will be doing a range of different fragment calculations, we'll give the fragment data directories a generic name, and will make use of the radical keyword. This time, we'll turn it into a function for easy reuse.

In [12]:
def setup_molecular_fragment_data(fragment_name, template_name):
    import os
    
    main_dir = os.getcwd()

    data_dir = 'data-'+fragment_name
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    os.chdir(data_dir)

    if not os.path.islink(template_name+'.xyz'):
        os.symlink('../'+template_name+'.xyz', template_name+'.xyz')

    if not os.path.islink('data-'+template_name):
        os.symlink('../data-'+template_name, 'data-'+template_name)

    os.chdir(main_dir)
    
fragment_name = 'fragments'
setup_molecular_fragment_data(fragment_name, template_name)

Finally, we'll also again make the fragment input file, using the same parameters as before.

In [13]:
inpf['dft'].update({'inputpsiid': 'linear_restart'})
inpf['perf'].update({'adjust_kernel_iterations': False, 'adjust_kernel_threshold': False})
inpf['lin_general'].update({'hybrid': False, 'nit': [0, 1], 'output_mat': 0, 'output_wf': 0,
                            'subspace_diag': True})
inpf.update({'lin_basis': {'nit': 1, 'idsx': 1}})
inpf.update({'lin_kernel': {'nit': 200, 'rpnrm_cv': 1.0e-11, 'delta_pnrm': -1, 'alphamix': 0.1}})

# as before, we have a single template type
inpf.update({'frag': {template_name: [i for i in range(1, nat + 1)]}})

inpf.update({'radical': fragment_name})

Let's now run the fragment calculation using the isolated C template.

In [14]:
# we'll label the run with both the system name and the template name
name = 'periodic_chain'+str(nat)+'_fragment_'+template_name

# we need to remember to use the fragment input file, instead of the template one
run_fragment = code.run(input=inpf, sys=periodic_chain, name=name)    

converged = check_convergence_fixed_sfs(run_fragment)

fragment_energy = Ha2eV * run_fragment.energy / run_fragment.nat
fragment_time = run_fragment.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Fragment calculation for the chain took '+'{0:.1f}'.format(fragment_time)+' minutes, E = '+\
      '{0:.3f}'.format(fragment_energy)+' eV/atom')

print('Difference between fragment and linear scaling energy = '+\
      '{0:.3f}'.format(fragment_energy - full_energy)+' eV/atom')

Fragment calculation for the chain took 0.4 minutes, E = -175.159 eV/atom
Difference between fragment and linear scaling energy = 1.618 eV/atom


Unlike the anthracene dimer, the difference between the fragment and linear scaling calculations is high - more than 1 eV/atom. This is unsurprising, since as we discussed, an isolated C atom is not representative of a C atom in a periodic chain. 

Instead, we need to somehow generate a template in an environment which is similar to the C atom in the chain. In other words, we need to _embed_ the template into a representative environment, which is what we will do in the following section.

### Embedded C Atom Template

We want the template calculation to have the following features:
- the template environment should match as close as possible to the target environment
- the system should be as small as possible, so that the fragment and template calculations still remain cheaper than doing a full linear scaling calculation

In this case, we can achieve both goals by using a smaller periodic C chain, with the same bond length. 

Linear scaling BigDFT has a lower limit on how small a system can be when using periodic boundary conditions, in that the support functions (plus the additional grid points used when applying the Hamiltonian) must be smaller than the axis. For the parameters we are using in this tutorial, this limit is around 10 atoms, so let's create a new chain containing 10 atoms.

In [15]:
# we can reuse the function we created earlier, changing only the number of atoms
nat_template = 10
short_periodic_chain = create_periodic_chain(bond_length, nat_template)

The template calculation uses a very similar input file to the isolated template calculation - we still perform a linear scaling calculation, and we still output the matrices and support functions. 

However, there is a key difference, in that we already need to tell BigDFT that the system is composed of fragments. This follows the same syntax as the molecular fragment calculation, but this time we also need to already setup the data directories.

In [16]:
# since we're creating the directories from scratch, we'll have to modify the function we created
def setup_embedded_fragment_data(fragment_name, template_name):
    import os
    
    main_dir = os.getcwd()

    data_dir = 'data-'+fragment_name
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    os.chdir(data_dir)

    if not os.path.islink(template_name+'.xyz'):
        os.symlink('../'+template_name+'.xyz', template_name+'.xyz')

    if not os.path.exists('data-'+template_name):
        os.makedirs('data-'+template_name)

    os.chdir(main_dir)

import shutil
    
# before we can generate the folders, we need to make sure we have the template xyz file
# the template itself will still be the same (a single C atom),
# but we'll give it another name to keep the data separate
embedded_template_name = 'embedded_C'
shutil.copy(template_name+'.xyz', embedded_template_name+'.xyz')
setup_embedded_fragment_data(fragment_name, embedded_template_name)

# first let's define the fragment list - as before we have a single template type
inpt.update({'frag': {embedded_template_name: [i for i in range(1, nat_template + 1)]}})

# we'll also tell BigDFT to only write the associated files in the fragment directories
# in other words the files should be written in data-fragments/data-template only,
# and not also directly in data-fragments
# this will save disk space
inpt['lin_general'].update({'output_fragments': 1})

# we also need to reactivate writing out the required files
inpt['lin_general'].update({'output_wf': 2, 'output_mat': 1})

inpt.update({'radical': fragment_name})

Now we have all the ingredients we need to run the embedded template calculation.

In [17]:
# we'll name the calculation after the number of atoms in the chain
name = 'periodic_chain'+str(nat_template)+'_template'

run_embedded_template = code.run(input=inpt, sys=short_periodic_chain, name=name)    

converged = check_convergence_optimised_sfs(run_embedded_template)

embedded_template_energy = Ha2eV * run_embedded_template.energy / run_embedded_template.nat
embedded_template_time = run_embedded_template.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Template chain took '+'{0:.1f}'.format(embedded_template_time)+' minutes, E = '+\
      '{0:.3f}'.format(embedded_template_energy)+' eV/atom')

Template chain took 0.9 minutes, E = -176.933 eV/atom


We can now run a fragment calculation using this new template. The only thing we have to change is the fragment name.

In [18]:
inpf.update({'frag': {embedded_template_name: [i for i in range(1, nat + 1)]}})

name = 'periodic_chain'+str(nat)+'_fragment_'+embedded_template_name

run_embedded_fragment = code.run(input=inpf, sys=periodic_chain, name=name)    

converged = check_convergence_fixed_sfs(run_fragment)

embedded_fragment_energy = Ha2eV * run_embedded_fragment.energy / run_embedded_fragment.nat
embedded_fragment_time = run_embedded_fragment.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Embedded fragment calculation for the chain took '+'{0:.1f}'.format(embedded_fragment_time)+\
      ' minutes, E = '+'{0:.3f}'.format(embedded_fragment_energy)+' eV/atom')

print('Difference between embedded fragment and linear scaling energy = '+\
      '{0:.3f}'.format(embedded_fragment_energy - full_energy)+' eV/atom')

Embedded fragment calculation for the chain took 0.4 minutes, E = -176.774 eV/atom
Difference between embedded fragment and linear scaling energy = 0.003 eV/atom


The error with respect to the full linear scaling calculation is now only a few meV/atom, confirming that the embedded template environment is a good representation of the target environment.

Beyond the general similarity between local chemical environments, we can also more directly quantify any distortions between the atomic structure of the template and target system. We can do this by extracting the average cost function value, $J$, defined as:

$J_{\alpha\beta}=\frac{1}{N_{\mathrm{at}}}\sum_{a=1}^{N_{\mathrm{at}}}||\mathbf{R}_a^\alpha-\sum_{b=1}^{N_{\mathrm{at}}}\mathcal{R}^{\beta\rightarrow \alpha}_{ab}\mathbf{R}_a^\beta||^2$,

where $\mathcal{R}^{\beta\rightarrow \alpha}_{ab}$ is a rotation matrix between two instances of a fragment characterised by their atomic coordinates $\mathbf{R}^{\alpha}$ and $\mathbf{R}^{\beta}$, and $N_{\mathrm{at}}$ is the number of atoms in the fragment. By definition, $J=0$ corresponds to a rigid rotation.

We can see below that $J_{av}$ is zero. However, this is only meaningful in the case where fragments contain more than 1 atom. 

In [19]:
J_av = run_embedded_fragment.log['Input Hamiltonian']['Average Wahba cost function value']
print('Average cost function value = '+'{0:.2f}'.format(J_av)+' angstroem')

Average cost function value = 0.00 angstroem


However, we can also calculate $J$ using PyBigDFT for an arbitrary set of coordinates. As an example, let's compare two C dimers with differing bond lengths.

As we can see, this difference in bond lengths leads to a non-zero cost function.

In [20]:
from BigDFT.Fragments import RotoTranslation

atom1 = Atom({"sym": "C", "r": [0.0, 0.0, 0.0], "units": "angstroem"})
atom2 = Atom({"sym": "C", "r": [0.0, 0.0, 1.5], "units": "angstroem"})
dimer1 = Fragment([atom1, atom2])

atom1 = Atom({"sym": "C", "r": [0.0, 0.0, 0.0], "units": "angstroem"})
atom2 = Atom({"sym": "C", "r": [0.0, 0.0, 1.6], "units": "angstroem"})
dimer2 = Fragment([atom1, atom2])

J = RotoTranslation(dimer1, dimer2).J

print('J='+'{0:.2f}'.format(J)+' angstroem')

J=0.09 angstroem


This error also depends on another factor, namely whether or not any reformatting of the support functions is required, i.e. in cases where the support functions need rotating or shifting, or other details of the grid change, then interpolation will need to be performed. This is what we saw in the molecular fragments tutorial, where the eggbox effect was visible.

In this case, we have a wavelet grid spacing of 0.5 bohr, which is not a factor of the bond length. We would therefore expect that the atoms do not align with the grid points, and interpolation should be required to shift the support functions.

We can check what reformatting changes are made by inspecting the log.

In [21]:
print('Support function reformatting:')
print(run_embedded_fragment.log['Input Hamiltonian']\
      ['Overview of the reformatting (several categories may apply)'])

Support function reformatting:
{'No reformatting required': 80, 'Grid spacing has changed': 0, 'Box size has changed': 0, 'Number of coarse grid points has changed': 0, 'Number of fine grid points has changed': 0, 'Molecule was shifted': 0, 'Molecule was rotated': 0, 'Wrapping/unwrapping': 76}


Other than wrapping/unwrapping due to the periodic boundary conditions, which does not require any interpolation, no reformatting has taken place.

This is surprising, however we are working in periodic boundary conditions. BigDFT has to modify the grid spacing so that the grid spacing is consistent with the cell dimensions along the periodic direction. If we inspect the logfile, we can see that the grid spacing has been modified in the $z$-direction, matching the cell dimensions and also resulting in the atoms being on grid points. This explains why no reformatting was required.

In [22]:
print('Grid spacings = ',run_embedded_fragment.log['Box Grid spacings'])

Grid spacings =  [0.5, 0.5, 0.4724]


## Finite Carbon Chain

Let's now look at a system where we have more than one type of template environment. One example of a system where this is required is a finite C chain, instead of a periodic one, where the edge atoms will experience a different environment to the central atoms. 

### System Construction

Let's first modify the function we created to make a finite chain. As before, we'll create a 20 atom chain which is our target system. We'll terminate each end with H.

In [23]:
def create_finite_chain(c_c_bond_length, c_h_bond_length, natC):
    
    finite_chain = System()
    
    # add the 1st H atom
    atom = Atom({"sym": "H", "r": [0.0, 0.0, 0.0], "units": "angstroem"})
    finite_chain["FRA:0"] = Fragment([atom])

    # add the C atoms
    x = c_h_bond_length
    for iat in range(natC):
        atom = Atom({"sym": "C", "r": [0.0, 0.0, x], "units": "angstroem"})
        finite_chain["FRA:"+str(iat + 1)] = Fragment([atom])
        if iat < natC - 1:
            x += c_c_bond_length
        else:
            x += c_h_bond_length
        
    # and now the final H atom
    atom = Atom({"sym": "H", "r": [0.0, 0.0, x], "units": "angstroem"})
    finite_chain["FRA:"+str(natC + 1)] = Fragment([atom])
    
    return finite_chain


bond_length = 1.5
c_h_bond_length = 1.1
nat = 20
finite_chain = create_finite_chain(bond_length, c_h_bond_length, nat)

As before, we'll run the full linear scaling calculation to give us a reference energy. We'll first need to remove the fragment definition from the input file, as well as turn off I/O.

In [24]:
name = 'finite_chain'+str(nat)+'_linear'

inpt.pop('frag')
inpt['lin_general'].update({'output_wf': 0, 'output_mat': 0})
inpt['lin_general'].update({'subspace_diag': True})

run_finite = code.run(input=inpt, sys=finite_chain, name=name)    

converged = check_convergence_optimised_sfs(run_finite)

finite_energy = Ha2eV * run_finite.energy / run_finite.nat
finite_time = run_finite.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Fully optimised finite chain took '+'{0:.1f}'.format(finite_time)+' minutes, E = '+\
      '{0:.3f}'.format(finite_energy)+' eV/atom')

Fully optimised finite chain took 2.2 minutes, E = -162.104 eV/atom


Since we're now working in free boundary conditions, we're no longer restricted by a minimum system size. However, for consistency, let's use a template calculation of the same size.

In [25]:
nat_template = 10
short_finite_chain = create_finite_chain(bond_length, c_h_bond_length, nat_template)

### Fragment Definitions

Now we have to make a choice about how to define our fragments. Let's assume a simple scenario - an edge-like region, and a bulk-like region. To begin with, we'll assume that the C atoms bonded to H are part of the edge region, and all other C atoms form part of the bulk. We will therefore define an edge fragment consisting of a C-H pair, and a bulk fragment, which consists of a single C atom.

When using the fragment approach in BigDFT, atoms need to be both contiguous and in the same order. Since the left edge fragment is HC and the right fragment is CH, this will cause a problem. We can either reorder the input file, or we can simply define two edge fragment types - a left and right edge.

We have to give these fragment types a name - let's just call them edgeL, edgeR and bulk for simplicity. We then need to set up the directories.

In [26]:
# the bulk fragment is just a single C atom, so we can copy the existing template file
bulk_template_name = 'bulk'
shutil.copy(template_name+'.xyz', bulk_template_name+'.xyz')
setup_embedded_fragment_data(fragment_name, bulk_template_name)

# the edge fragment requires a new xyz file
# we can copy the first two fragments from the template -
# since we're not explicitly using the fragment functionality of PyBigDFT,
# it doesn't matter that the system is made of more than one fragment
from copy import deepcopy
edgeL_template = System()
for ifrag in range(2):
    edgeL_template["FRA:"+str(ifrag)] = deepcopy(short_finite_chain["FRA:"+str(ifrag)])

# we can now write out the left edge template
edgeL_template_name = 'edgeL'
ofile = open(edgeL_template_name+'.xyz', 'w')
write_xyz(edgeL_template, ofile)
ofile.close()

edgeR_template = System()
for ifrag in range(nat_template, nat_template+2):
    edgeR_template["FRA:"+str(ifrag)] = deepcopy(short_finite_chain["FRA:"+str(ifrag)])

# we can now write out the right edge template
edgeR_template_name = 'edgeR'
ofile = open(edgeR_template_name+'.xyz', 'w')
write_xyz(edgeR_template, ofile)
ofile.close()

# and setup the associated directories
setup_embedded_fragment_data(fragment_name, edgeL_template_name)
setup_embedded_fragment_data(fragment_name, edgeR_template_name)

Finally, we again need to update the input file. This time, we have to do some extra work to define the two fragment types.

In [27]:
# the fragments are in the order edgeL, bulk, ..., bulk, edgeR
# therefore the bulk fragments go from 2, ..., natC - 1
# we can use this to build the fragment dictionary
inpt.update({'frag': {edgeL_template_name: [1],
                      bulk_template_name: [i for i in range(2, nat_template)],
                      edgeR_template_name: [nat_template]}})

inpt['lin_general'].update({'output_wf': 2, 'output_mat': 1})

We can now run the template calculation.

In [28]:
name = 'finite_chain'+str(nat_template)+'_template'

run_finite_template = code.run(input=inpt, sys=short_finite_chain, name=name)    

converged = check_convergence_optimised_sfs(run_finite_template)

finite_template_energy = Ha2eV * run_finite_template.energy / run_finite_template.nat
finite_template_time = run_finite_template.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Template chain took '+'{0:.1f}'.format(finite_template_time)+' minutes, E = '+\
      '{0:.3f}'.format(finite_template_energy)+' eV/atom')

Template chain took 1.4 minutes, E = -149.888 eV/atom


Let's also update the fragment dictionary for the fragment calculation.

In [29]:
inpf.update({'frag': {edgeL_template_name: [1],
                      bulk_template_name: [i for i in range(2, nat)],
                      edgeR_template_name: [nat]}})

We can now run the fragment calculation.

In [30]:
name = 'finite_chain'+str(nat)+'_fragment_edge_bulk'

run_finite_fragment = code.run(input=inpf, sys=finite_chain, name=name)    

converged = check_convergence_fixed_sfs(run_finite_fragment)

finite_fragment_energy = Ha2eV * run_finite_fragment.energy / run_finite_fragment.nat
finite_fragment_time = run_finite_fragment.log['Timings for root process']['Elapsed time (s)'] / 60.0   
                    
print('Fragment calculation for the chain took '+'{0:.1f}'.format(finite_fragment_time)+' minutes, E = '+\
      '{0:.3f}'.format(finite_fragment_energy)+' eV/atom')

print('Difference between fragment and linear scaling energy = '+\
      '{0:.3f}'.format(finite_fragment_energy - finite_energy)+' eV/atom')

Fragment calculation for the chain took 0.7 minutes, E = -161.830 eV/atom
Difference between fragment and linear scaling energy = 0.273 eV/atom


The error is much lower than the scenario where we used an isolated template for the periodic chain, but it's still higher than we would like. However, we made an assumption that only the C atoms bonded to H are edge-like, while all others are bulk-like. 

This is an over-simplification, since the next nearest neighbours may also feel the effect of the edges. In addition, BigDFT will write out the support functions for the first instance of a given fragment, which means that the bulk fragment will be coming from an atom quite close to the edge, which may again be a poor approximation. If we want to reduce the error, we therefore need to change the fragment setup.

There are a number of ways we could do this - actually running the calculations is beyond the scope of this tutorial, but to give some suggestions:
- You could change the fragment definition to add an extra C atom to the edge fragments, i.e. the left fragment would consist of H-C-C. In other words, we also treat the second nearest neighbouring C atoms to the H as distinct from the bulk-like C atoms. This will reduce the error.
- Note that this approach requires running both a new template and full system calculation. We could instead define each atom as a unique type of fragment. This would mean that we could perform a single template (short chain) calculation, and only repeat the full system (long chain) calculation for different splits of bulk and edge-like atoms
- It's also possible to mix and match fragments coming from different template calculations, so you could use a bulk fragment coming from the periodic calculation, and edge fragments coming from the finite template calculation.

## Using a Fragment Guess

Sometimes, the fragment approach (either molecular or embedded) isn't accurate enough for the system and quantity we are interested in. This could be because the fragments are too distorted (i.e. high $J$), too closely interacting, or the number of fragment types required would involve building too large a template system.

In these cases, we can either directly use LS-BigDFT, or we can use the fragment approach to build an input guess for LS-BigDFT, and further optimise the SFs from there. This follows the same principle as using LS-BigDFT to generate a cubic scaling guess, in fact if we wanted we could go from a fragment guess directly to cubic scaling BigDFT. For the final part of this tutorial, we will focus only on using a fragment guess for LS-BigDFT.

Let's start by copying the fragment input file, and then making some changes to the SF and kernel optimisation parameters.

In [31]:
inpr = deepcopy(inpf)

# we need to turn back on SF optimisation - let's set both the number of iterations and DIIS history length to 4
inpr['lin_basis'].update({'nit': 4, 'idsx': 4})

# we'll also modify the kernel optimisation loop to not fully converge each time
inpr['lin_kernel'].update({'nit': 6})

# finally we need to update the number of outer loop iterations
inpr['lin_general'].update({'nit': [0, 200]})

# everything else can stay the same,
# but let's set the fragment definition again, in case you overwrote it when performing the exercises

inpr.update({'frag': {edgeL_template_name: [1],
                      bulk_template_name: [i for i in range(2, nat)],
                      edgeR_template_name: [nat]}})

Now we can run the calculation.

In [32]:
name = 'finite_chain'+str(nat)+'_fragment_guess_edge_bulk'

run_finite_fragment_guess = code.run(input=inpr, sys=finite_chain, name=name)    

converged = check_convergence_fixed_sfs(run_finite_fragment_guess)

finite_fragment_guess_energy = Ha2eV * run_finite_fragment_guess.energy / run_finite_fragment_guess.nat
finite_fragment_guess_time = run_finite_fragment_guess.log['Timings for root process']['Elapsed time (s)'] / 60.0
                    
print('Fragment calculation took '+'{0:.1f}'.format(finite_fragment_time)+' minutes, E = '+\
      '{0:.3f}'.format(finite_fragment_energy)+' eV/atom')

print('Fragment guess calculation took '+'{0:.1f}'.format(finite_fragment_guess_time)+' minutes, E = '+\
      '{0:.3f}'.format(finite_fragment_guess_energy)+' eV/atom')

print('Full linear Scaling calculation took '+'{0:.1f}'.format(finite_time)+' minutes, E = '+\
      '{0:.3f}'.format(finite_energy)+' eV/atom')

print('')

print('Difference between fragment and linear scaling energy = '+\
      '{0:.3f}'.format(finite_fragment_energy - finite_energy)+' eV/atom')

print('Difference between fragment guess and linear scaling energy = '+\
      '{0:.3f}'.format(finite_fragment_guess_energy - finite_energy)+' eV/atom')

Fragment calculation took 0.7 minutes, E = -161.830 eV/atom
Fragment guess calculation took 1.6 minutes, E = -162.105 eV/atom
Full linear Scaling calculation took 2.2 minutes, E = -162.104 eV/atom

Difference between fragment and linear scaling energy = 0.273 eV/atom
Difference between fragment guess and linear scaling energy = -0.001 eV/atom


As we might expect, the fragment guess approach gives approximately the same energy as the linear scaling approach. The small difference is purely down to the different convergence routes, and should disappear as the convergence tolerances for the SFs, density kernel and outer loop are decreased.

Of course, the computational cost is also more expensive compare to doing a one-shot calculation. The cost of the template calculation also needs to be taken into account, while the quality of the guess may also affect how many iterations are needed to converge. However, in cases where the full system is much larger than the template, such an approach may prove to be advantageous compared to doing a LS calculation from scratch.